# Drug Drug Interaction 

For each entity in a sentence, see if it has interaction with another entitiy in the sentence and of which type.

## Table of contents

1. [Anonymizing the data](#1)
2. [Word embeding](#2)
3. [LSTM](#3)

In [27]:
import sys
from os import listdir
import os

from xml.dom.minidom import parse

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt

import sklearn
from sklearn import preprocessing

import gensim 
from gensim.models import Word2Vec

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation

print('The keras version is {}.'.format(keras.__version__))
print('The nltk version is {}.'.format(sklearn.__version__))
print('The gensim version is {}.'.format(gensim.__version__))

The keras version is 2.2.4.
The nltk version is 0.21.1.
The gensim version is 3.4.0.


In [2]:
dir_train_medline = "/Users/lluccardoner/PycharmProjects/MET_AHLT_Lab_2019/data/Train/MedLine/"
dir_train_drugbank = "/Users/lluccardoner/PycharmProjects/MET_AHLT_Lab_2019/data/Train/DrugBank/"

In [3]:
dir_test_medline = "/Users/lluccardoner/PycharmProjects/MET_AHLT_Lab_2019/data/Test-DDI/MedLine/"
dir_test_drugbank = "/Users/lluccardoner/PycharmProjects/MET_AHLT_Lab_2019/data/Test-DDI/DrugBank/"

## Anonymizing the data <a name="1"></a>

For each pair of entities in a sentence, sustitue the entity names with dummy names.

In [4]:
def get_dummy_text(txt, ents, e1, e2, error=False, debug=False):
    try:
        d = txt
        diff = 0
        for e in ents: 
            e_start = int(ents[e][0]) - diff
            e_end = int(ents[e][1]) - diff 
            e_len = e_end - e_start + 1

            if e==e1: 
                dummy = "drug01"
            elif e==e2:
                dummy = "drug02"
            else:
                dummy= "drug00"
            
            d = d[:e_start] + dummy + d[e_end+1:]
            if debug:
                print(diff, e_start, e_end, e_len)
                print(d, end = "\n\n")
            diff = diff  + (e_len - len(dummy))
        return d
    except Exception as ex:
        if error:
            print(e + ': ' + str(ex))

In [5]:
# Test the function
s_txt = "Pharmacokinetic properties of abacavir were not altered by the addition of either lamivudine or zidovudine or the combination of lamivudine and zidovudine." 
d = {'1': ['30', '37'], '2': ['82', '91'], '3': ['96', '105'], '4': ['129', '138'], '5': ['144', '153'], } 
e1 = "2"
e2 = "4"

get_dummy_text(s_txt, d, e1, e2)

'Pharmacokinetic properties of drug00 were not altered by the addition of either drug01 or drug00 or the combination of drug02 and drug00.'

In [6]:
pairs_list = []
for d in [dir_train_drugbank, dir_train_medline]:
    for f in listdir(d):
        tree = parse(d + "/" + f)
        sentences = tree.getElementsByTagName("sentence")
        for s in sentences:
            s_id = s.attributes["id"].value
            s_text = s.attributes["text"].value

            entities = {}
            ents = s.getElementsByTagName("entity")
            for e in ents:
                e_id = e.attributes["id"].value
                e_offs = e.attributes["charOffset"].value.split(";")[-1].split("-") # When two, offsets pick the second one
                entities[e_id] = e_offs

            pairs = s.getElementsByTagName("pair")
            for p in pairs:
                p_id = p.attributes["id"].value
                e1_id = p.attributes["e1"].value
                e2_id = p.attributes["e2"].value
                is_ddi = p.attributes["ddi"].value
                
                ddi = 1 if is_ddi=="true" else 0 
                ddi_type = p.attributes["type"].value if is_ddi=="true" and "type" in p.attributes else "none"
                
                s_dummy = get_dummy_text(s_text, entities, e1_id, e2_id)
                s_dummy = 'SOS '+s_dummy+ ' EOS'
                
                pair_item = {"p_id":p_id, "e1_id":e1_id, "e2_id":e2_id, "ddi":ddi, "ddi_type": ddi_type, 
                             "e1_offs":entities[e1_id], "e2_offs":entities[e2_id], "s_id":s_id, "s_text":s_text,
                             "s_dir":d, "set":d.split("/")[-2], "s_file":f,
                             "s_text_dummy": s_dummy}
                pairs_list.append(pair_item)
        
df_train = pd.DataFrame(pairs_list)

In [7]:
df_train[["ddi_type", "set"]].groupby(["set"])['ddi_type'].value_counts()

set       ddi_type 
DrugBank  none         22217
          effect        1535
          mechanism     1257
          advise         818
          int            178
MedLine   none          1555
          effect         152
          mechanism       62
          int             10
          advise           8
Name: ddi_type, dtype: int64

The parsing of the offset is not right for all elements. Some entities have an offset like [94, 97;114, 121]

In [8]:
df_train[df_train["e1_id"]=="DDI-DrugBank.d314.s30.e2"]

,ddi,ddi_type,e1_id,e1_offs,e2_id,e2_offs,p_id,s_dir,s_file,s_id,s_text,s_text_dummy,set
2317,0,none,DDI-DrugBank.d314.s30.e2,"[114, 121]",DDI-DrugBank.d314.s30.e3,"[102, 121]",DDI-DrugBank.d314.s30.p5,/Users/lluccardoner/PycharmProjects/MET_AHLT_L...,Dexamethasone_ddi.xml,DDI-DrugBank.d314.s30,Vaccines: Patients on corticosteroid therapy m...,SOS drug00: Patients on drug00 therapy may exh...,DrugBank


In [9]:
df_train[df_train['s_text_dummy'].isnull()].size

0

## Word embeding <a name="2"></a>

Train a word2vec model with the data

In [10]:
df_train['tokens'] = df_train['s_text_dummy'].apply(word_tokenize)

In [11]:
data = df_train['tokens'].values
data.size

27792

In [12]:
# https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec
model1 = Word2Vec(data, min_count = 1, size = 100, window = 5)

In [13]:
df_train['tokens_enc'] = df_train['tokens'].apply(lambda x: [model1.wv[word] for word in x])

In [14]:
df_train[['tokens', 'tokens_enc']].head(2)

,tokens,tokens_enc
0,"[SOS, Milk, ,, milk, products, ,, and, drug01-...","[[-1.4499204, -1.0749171, 0.59493756, -0.75476..."
1,"[SOS, The, concurrent, administration, of, dru...","[[-1.4499204, -1.0749171, 0.59493756, -0.75476..."


## LSTM <a name="3"></a>

Train a LSTM with the word vectors.

In [15]:
df_train.head(5)

,ddi,ddi_type,e1_id,e1_offs,e2_id,e2_offs,p_id,s_dir,s_file,s_id,s_text,s_text_dummy,set,tokens,tokens_enc
0,1,mechanism,DDI-DrugBank.d481.s0.e0,"[25, 31]",DDI-DrugBank.d481.s0.e1,"[82, 86]",DDI-DrugBank.d481.s0.p0,/Users/lluccardoner/PycharmProjects/MET_AHLT_L...,Estramustine_ddi.xml,DDI-DrugBank.d481.s0,"Milk, milk products, and calcium-rich foods or...","SOS Milk, milk products, and drug01-rich foods...",DrugBank,"[SOS, Milk, ,, milk, products, ,, and, drug01-...","[[-1.4499204, -1.0749171, 0.59493756, -0.75476..."
1,1,effect,DDI-DrugBank.d419.s0.e0,"[33, 43]",DDI-DrugBank.d419.s0.e1,"[49, 58]",DDI-DrugBank.d419.s0.p0,/Users/lluccardoner/PycharmProjects/MET_AHLT_L...,Clavulanate_ddi.xml,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...,SOS The concurrent administration of drug01 an...,DrugBank,"[SOS, The, concurrent, administration, of, dru...","[[-1.4499204, -1.0749171, 0.59493756, -0.75476..."
2,0,none,DDI-DrugBank.d419.s0.e0,"[33, 43]",DDI-DrugBank.d419.s0.e2,"[175, 184]",DDI-DrugBank.d419.s0.p1,/Users/lluccardoner/PycharmProjects/MET_AHLT_L...,Clavulanate_ddi.xml,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...,SOS The concurrent administration of drug01 an...,DrugBank,"[SOS, The, concurrent, administration, of, dru...","[[-1.4499204, -1.0749171, 0.59493756, -0.75476..."
3,0,none,DDI-DrugBank.d419.s0.e1,"[49, 58]",DDI-DrugBank.d419.s0.e2,"[175, 184]",DDI-DrugBank.d419.s0.p2,/Users/lluccardoner/PycharmProjects/MET_AHLT_L...,Clavulanate_ddi.xml,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...,SOS The concurrent administration of drug00 an...,DrugBank,"[SOS, The, concurrent, administration, of, dru...","[[-1.4499204, -1.0749171, 0.59493756, -0.75476..."
4,1,effect,DDI-DrugBank.d419.s1.e0,"[45, 54]",DDI-DrugBank.d419.s1.e1,"[73, 83]",DDI-DrugBank.d419.s1.p0,/Users/lluccardoner/PycharmProjects/MET_AHLT_L...,Clavulanate_ddi.xml,DDI-DrugBank.d419.s1,It is not known whether this potentiation of a...,SOS It is not known whether this potentiation ...,DrugBank,"[SOS, It, is, not, known, whether, this, poten...","[[-1.4499204, -1.0749171, 0.59493756, -0.75476..."


In [16]:
Ys = df_train['ddi_type']
Ys.shape

(27792,)

In [17]:
le = preprocessing.LabelEncoder()
le.fit(Ys)
print("Classes:",le.classes_)

Classes: ['advise' 'effect' 'int' 'mechanism' 'none']


In [18]:
df_train['ddi_type_enc'] = df_train['ddi_type'].apply(lambda x: le.transform([x]) )

In [46]:
df_train.head(1)

,ddi,ddi_type,e1_id,e1_offs,e2_id,e2_offs,p_id,s_dir,s_file,s_id,s_text,s_text_dummy,set,tokens,tokens_enc,ddi_type_enc
0,1,mechanism,DDI-DrugBank.d481.s0.e0,"[25, 31]",DDI-DrugBank.d481.s0.e1,"[82, 86]",DDI-DrugBank.d481.s0.p0,/Users/lluccardoner/PycharmProjects/MET_AHLT_L...,Estramustine_ddi.xml,DDI-DrugBank.d481.s0,"Milk, milk products, and calcium-rich foods or...","SOS Milk, milk products, and drug01-rich foods...",DrugBank,"[SOS, Milk, ,, milk, products, ,, and, drug01-...","[[-1.4499204, -1.0749171, 0.59493756, -0.75476...",[3]


In [19]:
Xs_enc = np.array(df_train['tokens_enc'].tolist())
Ys_enc = np.array(df_train['ddi_type_enc'].values)
print(Xs_enc.shape, Ys_enc.shape)

(27792,) (27792,)


In [20]:
# Xs_enc = [sentence1 , sentence2, ...]
# Xs_enc[i] = [word1, word2, ...]
# Xs_enc[i][j] = [vector]
# Xs_enc = [[[vector], [vector], ...] , [[vector], [vector], ...], ...]
# reshape input to be [sentence, words, features]
print(len(Xs_enc), len(Xs_enc[0]), len(Xs_enc[0][0]))

27792 19 100


In [21]:
sentence_len = [len(x) for x in Xs_enc]
print("Max:", max(sentence_len), "Min:", min(sentence_len), "Mean", np.mean(sentence_len))

Max: 152 Min: 3 Mean 54.08822682786413


In [23]:
trainX = np.zeros([27792, 152, 100])

for i, s in enumerate(Xs_enc):
    s = np.array(s)
    shape = s.shape
    for j in range(shape[0], 152):
        s = np.append(s, np.zeros([1,100]), axis=0)
    trainX[i] = s
    
print( type(trainX), trainX.shape)

In [44]:
to_categorical(5)

array([0., 0., 0., 0., 0., 1.], dtype=float32)

In [50]:
from keras.utils import to_categorical

trainY = np.zeros([27792, 5]) #5 classes

for i, l in enumerate(Ys_enc):
    print(to_categorical(l[0]))
    break
    #trainY[i] = np.array(to_categorical(l[0]))
    
print(type(trainY), trainY.shape)

[0. 0. 0. 1.]
<class 'numpy.ndarray'> (27792, 5)


In [34]:
model = Sequential()
model.add(LSTM(52, input_shape=(152, 100)))
model.add(Dense(1))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
history = model.fit(trainX, trainY, epochs=1, batch_size=100, verbose=1)

ValueError: You are passing a target array of shape (27792, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.